# Jupyter example for generation with `spcm`
_Spectrum Instrumentation GmbH (c)_

**2_jup_gen_single.py**

Shows a simple standard mode example using only the few necessary commands.
* There will be a saw-tooth signal generated on channel 0.
* This signal will have an amplitude of 1 V.

Example for analog replay cards (AWG) for the the M2p, M4i, M4x and M5i card-families.

_See the README file in the parent folder of this examples directory for information about how to use this example._

_See the LICENSE file for the conditions under which this software may be used and distributed._

In [ ]:
import spcm
from spcm import units # spcm uses the pint library for unit handling (units is a UnitRegistry object)

import numpy as np

## Open the card and setup the acquisition

* if you want to open a specific card using the device identifier

In [2]:
# card = spcm.Card('/dev/spcm0', verbose=True).open()

* if you want to open a remote card with ip-address

In [3]:
# card = spcm.Card('TCPIP::192.168.1.10::inst0::INSTR', verbose=True).open()

* if you want to open a card by its serial number

In [4]:
# card = spcm.Card(serial_number=12345, verbose=True).open()

* if you want to open the first card of a specific type

In [5]:
card = spcm.Card(card_type=spcm.SPCM_TYPE_AO, verbose=True).open()

Python version: 3.11.9 on Windows
Driver version: 7.7.23410
Kernel version: 0.0.0
Found '/dev/spcm0': M4i.6620-x8 sn 100000


### Setup the card to do data acquisition

For the acquisition, we set the operation mode of the card to `SPC_REP_STD_CONTINUOUS`. For more information about this mode we refer to the specific manual of your device. The card timeout is set to 10 s, this means that if it takes longer than 5 seconds to acquire the data after the card was started there will be an interrupt and the `spcm` package will throw an `spcm.SpcmTimeout` exception.

In [ ]:
card.card_mode(spcm.SPC_REP_STD_CONTINUOUS)
card.timeout(10 * units.s); # 10 s; 0 = disable timeout functionality
card.loops(0); # 0 = loop endless; >0 = n times

0

### Setup the input channels

Set up the usage of one channel (Channel 0) in the range of +/- 1000 mV. Electrical load of the channel is set to 50 Ohm.

In [ ]:
channels = spcm.Channels(card, card_enable=spcm.CHANNEL0)
channels.enable(True);
channels.output_load(units.highZ);
channels.amp(1 * units.V);

### Setup the clock

The clock is set to using the internal oscillator with PLL to generate the required `10%` of the maximum sampling rate. The clock output is disabled.

In [ ]:
clock = spcm.Clock(card)
clock.sample_rate(10 * units.percent);
clock.clock_output(False);

0

### Setup trigger

We'll be using the software trigger. This trigger is executed directly after the start of the card and enabling the trigger engine.

In [ ]:
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_SOFTWARE);

1

### Define the buffer for data transfer from the PC to the card and start the transfer

Setup a buffer in RAM to generate 1 MebiSamples (1024*1024 Samples) of data. The DMA transfer is started.

In [10]:
num_samples = 1 * units.MiS

data_transfer = spcm.DataTransfer(card)
if data_transfer.bytes_per_sample != 2: raise spcm.SpcmException(text="Non 16-bit DA not supported")
data_transfer.memory_size(num_samples) # size of memory on the card
data_transfer.allocate_buffer(num_samples) # size of buffer in pc RAM

# generate output data (or alternatively load data from file)
num_samples_magnitude = num_samples.to_base_units().magnitude
# simple ramp for analog output cards
data_transfer.buffer[:] = np.arange(-num_samples_magnitude//2, num_samples_magnitude//2).astype(np.int16) # saw-tooth signal

data_transfer.start_buffer_transfer(spcm.M2CMD_DATA_STARTDMA, spcm.M2CMD_DATA_WAITDMA) # Wait until the writing to buffer has been done

Starting the DMA transfer and waiting until data is in board memory
... data transfer started


In [11]:
print("Starting the card and waiting for ready interrupt\n(continuous and single restart will have timeout)")
try:
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_WAITREADY)
except spcm.SpcmTimeout as timeout:
    print("-> The 10 seconds timeout have passed and the card is stopped")

# Without the above "spcm.M2CMD_CARD_WAITREADY" flag you can do things here in parallel
# and later stop the replaying with "card.stop()"

Starting the card and waiting for ready interrupt
(continuous and single restart will have timeout)
-> The 10 seconds timeout have passed and the card is stopped
